In [ ]:
'''
    =================================================
   
    Program ini dibuat untuk melakukan automatisasi pengolahan (cleaning) data text yang berguna untuk pemodelan model analisa sentimen.
    =================================================
'''

##### **A. WEB SCRAPING** 
 
 In order to start the analyzation about the worth of selling 'Seblak' we need to get the data of the product's sales. The data will be taken from Tokopedia is :

 1.  `Nama Produk`
 2. `Harga Produk`
 3. `Penjual` 
 4. `Kota Toko`
 5. `Banyaknya Terjual`
 6. `Rating Produk` 

 By web scrapping from the first 10 pages of the web to get min 50 datas

In [13]:
#Importing all needed library

from selenium import webdriver
from bs4 import BeautifulSoup
from time import sleep

In [14]:
#install webdriver
driver = webdriver.Chrome()

In [15]:
#mengakses website 
url = 'https://www.tokopedia.com/search?navsource=&page=1&q=seblak&srp_component_id=02.01.00.00&srp_page_id=&srp_page_title=&st='
driver.get(url)

In [16]:
#mengakses HTML halaman 
html = driver.page_source  
soup = BeautifulSoup(html, 'html.parser')  

In [30]:
nama = []
price = []
seller = []
city = []
sold = []
rating = []
for i in range(1,11):
    url=f'https://www.tokopedia.com/search?navsource=&page={i}&q=seblak&srp_component_id=02.01.00.00&srp_page_id=&srp_page_title=&st='.format(i)
    driver.get(url)
    sleep(3)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    
    # mengumpulkan item 
    boxes = soup.find_all('div', {'class':"css-uwyh54"})
    
    for box in boxes :

        #mengambil nama
        element_nama = box.find('div', {'class':'prd_link-product-name css-3um8ox'})
        nama.append(element_nama.get_text()) 

        
        #mengambil harga
        element_harga = box.find('div', {'class':'prd_link-product-price css-h66vau'})
        price.append(element_harga.get_text()) 


        #mengambil nama penjual
        kumpulan_seller =box.find('span', {'class':'prd_link-shop-name css-1kdc32b flip'})
        try : 
            element_seller = kumpulan_seller.get_text()
            seller.append(element_seller)
        
        except : 
            seller.append(None)


        #mengambil kota lokasi
        kumpulan_city = box.find('span', {'class':'prd_link-shop-loc css-1kdc32b flip'})
        try : 
            element_city = kumpulan_city.get_text()
            city.append(element_city)
        except : 
            city.append(None)

        
        #mengambil sold item
        kumpulan_sold = box.find('span', {'class':'prd_label-integrity css-1sgek4h'})
        try : 
            element_sold = kumpulan_sold.get_text()
            sold.append(element_sold)

        except : 
            sold.append(0)

        
        #mengambil rating 
        kumpulan_rating = box.find('span', {'class':'prd_rating-average-text css-t70v7i'})
        try : 
            element_rating = kumpulan_rating.get_text()
            rating.append(element_rating)

        except : 
            rating.append (0)
        

    ## Catatan :
        # Kenapa untuk element seller dan city di tambah if statement?
        # Karena kalau dicheck ke laman tokopedia ada 3 produk yang tidak ada label seller dan kota nya (produk paling atas muncul)
        # Jadi untuk 3 produk tersebut tidak ada label kota dan sellernya. Kalau tidak pakai if, akan error pada saat program mendeteksi
        # kalau tidak ditemukan label seller dan kota. Nah, untuk produk-produk yang tidak ada info seller dan kotanya value nya diisi None aka tidak ditemukan.


In [3]:
import pandas as pd

In [33]:
data = pd.DataFrame()

data['nama produk'] = nama
data['harga'] = price
data['seller'] = seller
data['lokasi'] = city
data['terjual'] = sold
data['rating'] = rating

data

,nama produk,harga,seller,lokasi,terjual,rating
0,"Seblak Rafael, Seblak Coet Instan Halal",Rp25.000,Brother Meat Shop,Depok,250+ terjual,4.9
1,Seblak Instan,Rp8.000,ROEMAH MOSLEM,Kab. Boyolali,50+ terjual,5.0
2,KERUPUK MENTAH KERUPUK SEBLAK MENTAH 500 GR,Rp12.500,BociKakang,Jakarta Selatan,100+ terjual,4.9
3,Kerupuk Bawang Prima / Krupuk Seblak 500gram,Rp17.500,Parengge Rengge bumbu & rempah,Jakarta Timur,5 terjual,5.0
4,KERUPUK SEBLAK PEDAS DAUN JERUK 100 GR,Rp4.500,BociKakang,Jakarta Selatan,500+ terjual,4.8
...,...,...,...,...,...,...
345,seblak kering pedas 250 gr/seblak kencur/kerup...,Rp9.999,aghniya,Tangerang Selatan,500+ terjual,4.8
346,Gelifood Seblak Instan Kerupuk Mawar Bumbu Ken...,Rp15.000,Lidigeli,Kab. Garut,250+ terjual,4.9
347,keripik beling/kaca 1/2 kg. bumbu seblak daun ...,Rp25.999,aghniya,Tangerang Selatan,250+ terjual,4.9
348,Seblak Instan Pedas Home Made,Rp3.500,the Dhecip,Tangerang Selatan,3rb+ terjual,4.9


In [34]:
data.to_csv('dataseblak.csv', index = False)

In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   nama produk  350 non-null    object
 1   harga        350 non-null    object
 2   seller       350 non-null    object
 3   lokasi       350 non-null    object
 4   terjual      350 non-null    object
 5   rating       350 non-null    object
dtypes: object(6)
memory usage: 16.5+ KB


In [36]:
data.head ()

,nama produk,harga,seller,lokasi,terjual,rating
0,"Seblak Rafael, Seblak Coet Instan Halal",Rp25.000,Brother Meat Shop,Depok,250+ terjual,4.9
1,Seblak Instan,Rp8.000,ROEMAH MOSLEM,Kab. Boyolali,50+ terjual,5.0
2,KERUPUK MENTAH KERUPUK SEBLAK MENTAH 500 GR,Rp12.500,BociKakang,Jakarta Selatan,100+ terjual,4.9
3,Kerupuk Bawang Prima / Krupuk Seblak 500gram,Rp17.500,Parengge Rengge bumbu & rempah,Jakarta Timur,5 terjual,5.0
4,KERUPUK SEBLAK PEDAS DAUN JERUK 100 GR,Rp4.500,BociKakang,Jakarta Selatan,500+ terjual,4.8


#### **B. DATA PREPARATION** 

 Melakukan pengecekan data dan melakukan cleansing data. Memastikan bahwa data sudah tidak ada lagi missing value dan tipe data sudah sesuai dengan tipe data yang dibutuhkan untuk analisis.

In [6]:
data = pd.read_csv('dataseblak.csv')

data_seblak = data.copy()

In [7]:
data_seblak.sample(15)

,nama produk,harga,seller,lokasi,terjual,rating
219,cuanki/cuankie lidah 100 pcs termurah khas Gar...,Rp32.999,ArfaOnlineShop,Tangerang,23 terjual,5.0
210,Seblak Kuah Instant Terciduk Khas Bandung Extr...,Rp18.700,Gerai Snack Official Shop,Kab. Tangerang,60+ terjual,4.5
5,"Seblak Rafael, Seblak Coet Instan Halal",Rp25.000,Brother Meat Shop,Depok,250+ terjual,4.9
35,kerupuk seblak mentah,Rp5.000,Sembakofariz,Bandung,2 terjual,5.0
335,SAMBAL SEBLAK BY MAMANG RAFAEL ORIGINAL HALAL ...,Rp39.000,Beauty DELUXE,Jakarta Barat,17 terjual,5.0
337,Kerupuk Seblak Rafael Pedas / Seblak Mawar Ped...,Rp15.000,DUO BOCIL SNACK,Depok,80+ terjual,5.0
316,KERUPUK PEDAS KRIWIL BANTAT ISI 250gr KRITING ...,Rp17.248,AHR Jaya Snack,Jakarta Timur,60+ terjual,5.0
57,Kerupuk Seblak Rafael Pedas / Seblak Mawar Ped...,Rp15.000,DUO BOCIL SNACK,Depok,80+ terjual,5.0
25,Seblak Instan Pedas Home Made,Rp3.500,the Dhecip,Tangerang Selatan,3rb+ terjual,4.9
69,seblak / paket seblak bumbu komplit pedas ekon...,Rp3.479,Grosir Putra Bdg,Kab. Bandung,250+ terjual,4.8


In [8]:
data_seblak.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   nama produk  350 non-null    object 
 1   harga        350 non-null    object 
 2   seller       350 non-null    object 
 3   lokasi       350 non-null    object 
 4   terjual      350 non-null    object 
 5   rating       350 non-null    float64
dtypes: float64(1), object(5)
memory usage: 16.5+ KB


notes : data pada kolom harga dan terjual harus diganti tipe data interger karena nantinya akan butuh untuk dikalkulasi

In [9]:
data_seblak.isna().sum()

nama produk    0
harga          0
seller         0
lokasi         0
terjual        0
rating         0
dtype: int64

dari data di atas bisa dilihat bahwa sudah tidak ada null pada tabel. Maka, selanjutnya kita akan melakukan memperbaiki format dan mengubah tipe data sesuai dengan yang dibutuhkan 

In [10]:
# melihat sample data secara acak 
data_seblak.sample(15)

,nama produk,harga,seller,lokasi,terjual,rating
2,KERUPUK MENTAH KERUPUK SEBLAK MENTAH 500 GR,Rp12.500,BociKakang,Jakarta Selatan,100+ terjual,4.9
127,Kerupuk Seblak Rafael Pedas / Seblak Mawar Ped...,Rp15.000,DUO BOCIL SNACK,Depok,80+ terjual,5.0
216,kerupuk seblak pedas 1 bungkus,Rp10.900,atwshop,Jakarta Selatan,14 terjual,5.0
1,Seblak Instan,Rp8.000,ROEMAH MOSLEM,Kab. Boyolali,50+ terjual,5.0
84,"Kylafood Seblak Cup Instan, Halal",Rp11.500,Brother Meat Shop,Depok,80+ terjual,5.0
91,"Seblak Rafael, Seblak Coet Instan Halal",Rp25.000,Brother Meat Shop,Depok,250+ terjual,4.9
233,Seblak RAFAEL | Seblak MAMANG RAFAEL | Seblak ...,Rp39.500,Beauty DELUXE,Jakarta Barat,25 terjual,5.0
213,Kerupuk seblak - cemilan murah kaleastores,Rp13.500,kaleastores,Kab. Banyuwangi,0,0.0
74,Bundle - Basranger Seblak | Baso Aci Extra Ped...,Rp56.300,Rak Sebelah,Jakarta Pusat,100+ terjual,4.9
219,cuanki/cuankie lidah 100 pcs termurah khas Gar...,Rp32.999,ArfaOnlineShop,Tangerang,23 terjual,5.0


**Mengubah tipe data dan menghilangkan unnecessary values pada data**


In [11]:
# Mengubah 
data_seblak["nama produk"] = data_seblak["nama produk"].str.title()
data_seblak["harga"] = data_seblak["harga"].str.replace('Rp','').str.replace('.','').astype(int)
data_seblak["seller"] = data_seblak["seller"].str.title()
data_seblak["terjual"] = data_seblak["terjual"].str.replace('+','').str.replace('terjual','').str.replace('rb','000').astype(int)



C:\Users\ACER\AppData\Local\Temp\ipykernel_15232\3952105217.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data_seblak["harga"] = data_seblak["harga"].str.replace('Rp','').str.replace('.','').astype(int)
C:\Users\ACER\AppData\Local\Temp\ipykernel_15232\3952105217.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data_seblak["terjual"] = data_seblak["terjual"].str.replace('+','').str.replace('terjual','').str.replace('rb','000').astype(int)


In [12]:
data_seblak


,nama produk,harga,seller,lokasi,terjual,rating
0,"Seblak Rafael, Seblak Coet Instan Halal",25000,Brother Meat Shop,Depok,250,4.9
1,Seblak Instan,8000,Roemah Moslem,Kab. Boyolali,50,5.0
2,Kerupuk Mentah Kerupuk Seblak Mentah 500 Gr,12500,Bocikakang,Jakarta Selatan,100,4.9
3,Kerupuk Bawang Prima / Krupuk Seblak 500Gram,17500,Parengge Rengge Bumbu & Rempah,Jakarta Timur,5,5.0
4,Kerupuk Seblak Pedas Daun Jeruk 100 Gr,4500,Bocikakang,Jakarta Selatan,500,4.8
...,...,...,...,...,...,...
345,Seblak Kering Pedas 250 Gr/Seblak Kencur/Kerup...,9999,Aghniya,Tangerang Selatan,500,4.8
346,Gelifood Seblak Instan Kerupuk Mawar Bumbu Ken...,15000,Lidigeli,Kab. Garut,250,4.9
347,Keripik Beling/Kaca 1/2 Kg. Bumbu Seblak Daun ...,25999,Aghniya,Tangerang Selatan,250,4.9
348,Seblak Instan Pedas Home Made,3500,The Dhecip,Tangerang Selatan,3000,4.9


In [13]:
# menyimpan data yang bersih dalam csv

data_seblak.to_csv('clean_data_seblak.csv', index =False)

**C. Business Understanding/Problem Statement**

 **Menjabarkan problem statement menggunakan SMART framework berikut dengan penjabaran Specific, Measurable, Achievable, Relevant, dan Time-bound untuk menyelesaikan masalah dan menarik kesimpulan untuk "How To" achieve the goals**


Case : Saya ingin mendapatkan pendapatan tambahan selain dari pendapatan tetap saya. Saya berpikir untuk mendapatkannya dengan cara menjadi dropshipper produk seblak di Tokopedia. Saya ingin mengeksekusi bisnis dropshipping saya dengan pertimbangan yang matang dan menjadi bisnis yang berorientasi pada customer preference dalam pengambilan keputusan dan proses berjalannya. 

**SMART**

**Specific**        = Melakukan analisa dan mengambil keputusan tentang mendapatkan pendapatan tambahan dengan cara berhasil berjualan seblak melalui Tokopedia sebagai dropshipper 



**Measureable**     = Mendapatkan pandapatan tambahan dengan menjadi dropshipper produk seblak di Tokopedia sebesar 100.000 dalam sebulan 

**Achievable**      = Melakukan penjualan sebagai dropshipper seblak yang proses serta pengambilan keputusan proses bisnisnya mulai dari penjualannya serta pemilihan produk dan produsennya berdasarkan analisa minat konsumen 

**Relevant**        = mengetahui banyaknya minat konsumen untuk membeli seblak dan preferensi konsumennya, untuk bisa mendapatkan pendapatan sebagai dropshipper seblak yang terukur

**Time-bound**      = Melakukan analisa animo dan preferensi pembeli seblak di Tokopedia dalam waktu 1 minggu dan   kemudian mengambil keputusan tentang memulai atau tidaknya berjualan sebagai dropshipper seblak di Tokopedia dan 
apabila iya, melangsungkan proses berjualan dan evaluasinya selama 3 minggu sampai ke tahap evaluasi selanjutnya




**Problem Statement : Mampu mengambil keputusan untuk memulai langkah mendapatkan pendapatan tambahan setidaknya  sebesar 100.000 rupiah dengan menjalankan bisnis dropshipping produk seblak di Tokopedia yang berorientasi kepada customer, dimana setiap pengambilan keputusan bisnis (mulai dari analisa pengambilan keputusan dimulai atau tidaknya bisnis hingga sampai proses berjalannya bisnis) berlandaskan analisis preferensi pembeli produk seblak di Tokopedia yang akan dieksekusi dan dievaluasi dalam waktu 1 bulan hingga ke evaluasi berikutnya.**




##### **D. Data Analysis**

1. Menghitung nilai rata-rata, median, standar deviasi, skewness, dan kurtosis dari kolom harga, banyak produk terjual, dan rating serta penjabaran dari hasil data yang didapatkan 

In [4]:
#Membuat copy-data seblak yang clean
original_data_seblak = pd.read_csv('clean_data_seblak.csv')
analysis_data_seblak = original_data_seblak.copy()

In [15]:
analysis_data_seblak

,nama produk,harga,seller,lokasi,terjual,rating
0,"Seblak Rafael, Seblak Coet Instan Halal",25000,Brother Meat Shop,Depok,250,4.9
1,Seblak Instan,8000,Roemah Moslem,Kab. Boyolali,50,5.0
2,Kerupuk Mentah Kerupuk Seblak Mentah 500 Gr,12500,Bocikakang,Jakarta Selatan,100,4.9
3,Kerupuk Bawang Prima / Krupuk Seblak 500Gram,17500,Parengge Rengge Bumbu & Rempah,Jakarta Timur,5,5.0
4,Kerupuk Seblak Pedas Daun Jeruk 100 Gr,4500,Bocikakang,Jakarta Selatan,500,4.8
...,...,...,...,...,...,...
345,Seblak Kering Pedas 250 Gr/Seblak Kencur/Kerup...,9999,Aghniya,Tangerang Selatan,500,4.8
346,Gelifood Seblak Instan Kerupuk Mawar Bumbu Ken...,15000,Lidigeli,Kab. Garut,250,4.9
347,Keripik Beling/Kaca 1/2 Kg. Bumbu Seblak Daun ...,25999,Aghniya,Tangerang Selatan,250,4.9
348,Seblak Instan Pedas Home Made,3500,The Dhecip,Tangerang Selatan,3000,4.9


In [16]:
#importing needed library
from scipy import stats
import numpy as np

In [6]:
#check nilai mean, median, dan standard deviation pada kolom harga, banyaknya terjual, dan rating
#check skewness dan kurtosis untuk melihat kecengerungan dan distribusi data dan mengetahui adanya outlier


print (analysis_data_seblak.describe(),'\n')

#Pada kolom Harga
print('Median in Price :', analysis_data_seblak['harga'].median())
print('Skewness in Price :', analysis_data_seblak['harga'].skew())
print('Kurtosis in Price :', analysis_data_seblak['harga'].kurtosis(),'\n')

#pada kolom terjual
print('Median in Terjual :', analysis_data_seblak['terjual'].median())
print('Skewness in Terjual :', analysis_data_seblak['terjual'].skew())
print('Kurtosis in Terjual :', analysis_data_seblak['terjual'].kurtosis(),'\n')

#pada kolom rating 
print('Median in Rating :', analysis_data_seblak['rating'].median())
print('Skewness in Rating :', analysis_data_seblak['rating'].skew())
print('Kurtosis in Rating :', analysis_data_seblak['rating'].kurtosis(),'\n')


               harga       terjual      rating
count     350.000000    350.000000  350.000000
mean    17492.765714   1126.160000    4.781143
std     19816.535771   2463.637277    0.741437
min      3479.000000      0.000000    0.000000
25%      4500.000000     80.000000    4.900000
50%     12250.000000    250.000000    4.900000
75%     25000.000000    500.000000    5.000000
max    209280.000000  10000.000000    5.000000 

Median in Price : 12250.0
Skewness in Price : 5.4818371894673
Kurtosis in Price : 45.68622960892626 

Median in Terjual : 250.0
Skewness in Terjual : 2.969805645824807
Kurtosis in Terjual : 7.84213192238289 

Median in Rating : 4.9
Skewness in Rating : -6.184564159128454
Kurtosis in Rating : 37.313407485859635 



**Kesimpulan Kolom Harga**

Dapat dilihat dari hasil describe di atas bahwa nilai rata rata harga produk seblak adalah sebesar 17.492, dan untuk nilai tengah harga dari produk tersebut adalah 12.250. Nilai standar deviasi dari kolom harga adalah sebesar 19816. Melalui data di atas juga dapat dilihat bahwa nilai terendah harga produk seblak adalah senilai 3479 dan harga tertinggi adalah 209.280. Nilai skewness pada kolom harga berada jauh di atas 0, yaitu di angka 5.48 yang mengindikasikan distribusi data tidak normal dan positive skew. Nilai kurtosis dalam kolom price juga menunjukan nilai yang tinggi yaitu di nilai 45.68, hal ini menunjukkan bahwa distribusi data memiliki tingkat puncak yang sangat ekstrem, dan berbeda secara signifikan dengan mesokurtik, nilai tersebut mengindikasikan adanya kondisi ekstrem leptokurtik. Hal terebut menunjukkan bahwa terdapat data harga outlier yang sangat besar.




##### **Kesimpulan Kolom Terjual**

Dapat dilihat dari hasil describe kolom terjual di atas bahwa nilai rata rata produk seblak terjual adalah sebesar 350, dan untuk nilai tengah jumlah terjual dari produk tersebut adalah 250. Nilai standar deviasi dari kolom terjual adalah sebesar 2463.63. Melalui data di atas juga dapat dilihat bahwa jumlah terendah terjualnya produk seblak adalah 0 dan paling banyak terjual adalah 10000. Nilai skewness pada kolom terjual berada di atas 0, yaitu di angka 2.96 yang mengindikasikan distribusi data tidak normal dan positive skew. Nilai kurtosis dalam kolom terjual juga menunjukan nilai yang tinggi, yaitu di angka 7.84  nilai tersebut mengindikasikan adanya kondisi leptokurtik. Hal terebut menunjukkan bahwa terdapat data terjual outlier.



##### **Kesimpulan Kolom Rating**

Melalui data di atas dapat disimpulkan bahwa dari 350 produk seblak yang dijual di Tokopedia hasil nilai median atau nilai tengah pada rating dari pembeli produk seblak di Tokopedia menampilkan nilai yang lebih besar(4.9) dibandingkan mean atau nilai rata ratanya (4.7). Hal ini mengindikasikan bahwa hampir sebagian besar produk seblak yang dibeli di Tokopedia mendapatkan rating besar di atas rata rata (terlihat dari nilai median yang cenderung ditarik ke atas) dan hanya sebagian kecil yang mendapatkan rating rendah(dibawah rata-rata), dapat dikatakan apabila rating adalah parameter kepuasan pembeli dalam experience pembelian produk, banyak customer yang merasa puas dengan memberikan rating yang tinggi di produknya. Namun, nilai median (4.9) berada di atas nilai mean (4.7), yang menunjukkan ada beberapa nilai ekstrem yang lebih rendah yang mempengaruhi nilai mean. Skewness yang sangat negatif (-6) menunjukkan bahwa distribusi data sangat condong ke kiri dan ekstrem di sebelah kiri distribusi, sehingga nilai-nilai ekstrem tersebut jauh di bawah rata-rata dan median. Hal ini menunjukkan adanya penumpukan nilai di bagian atas distribusi. Jadi dapat disimpulkan bahwa dari 350 data seblak yang ada itu banyak produk yang memliki rating yang nialinya ekstrem di bawah rata rata dan median, atau dalam kasus rating e-commerce bisa juga ini adalah indikasi bahwa banyak produk yang belum di rating. Nilai kurtosis dalam data rating (37.3) menunjukkan bahwa data memiliki ekor yang panjang dan puncak yang sangat tinggi yang menunjukan bahwa nilai menunjukkan lebih banyak nilai ekstrem dibandingkan dengan nilai distribusi normal. Kesimpulan dari seluruh data pada kolom rating adalah distribusi data memiliki sifat yang sangat ekstrem dan cenderung condong ke kiri, dengan sebagian besar nilai yang tinggi dan beberapa nilai ekstrem yang rendah.


In [20]:
analysis_data_seblak

,nama produk,harga,seller,lokasi,terjual,rating,area
0,"Seblak Rafael, Seblak Coet Instan Halal",25000,Brother Meat Shop,Depok,250,4.9,JABODETABEK
1,Seblak Instan,8000,Roemah Moslem,Kab. Boyolali,50,5.0,NOT JABODETABEK
2,Kerupuk Mentah Kerupuk Seblak Mentah 500 Gr,12500,Bocikakang,Jakarta Selatan,100,4.9,JABODETABEK
3,Kerupuk Bawang Prima / Krupuk Seblak 500Gram,17500,Parengge Rengge Bumbu & Rempah,Jakarta Timur,5,5.0,JABODETABEK
4,Kerupuk Seblak Pedas Daun Jeruk 100 Gr,4500,Bocikakang,Jakarta Selatan,500,4.8,JABODETABEK
...,...,...,...,...,...,...,...
345,Seblak Kering Pedas 250 Gr/Seblak Kencur/Kerup...,9999,Aghniya,Tangerang Selatan,500,4.8,JABODETABEK
346,Gelifood Seblak Instan Kerupuk Mawar Bumbu Ken...,15000,Lidigeli,Kab. Garut,250,4.9,NOT JABODETABEK
347,Keripik Beling/Kaca 1/2 Kg. Bumbu Seblak Daun ...,25999,Aghniya,Tangerang Selatan,250,4.9,JABODETABEK
348,Seblak Instan Pedas Home Made,3500,The Dhecip,Tangerang Selatan,3000,4.9,JABODETABEK


2. Menentukan jumlah minimum dan maksimum dari hasil penjualan seblak melalui tokopedia 


In [14]:

from scipy import stats
import numpy as np

value = (analysis_data_seblak['harga'] * analysis_data_seblak['terjual'])
N = len(analysis_data_seblak)
low, up = stats.norm.interval(0.95,loc=value.mean(),scale=value.std()/np.sqrt(N))
print('Lower Limit:',low)
print('Upper Limit:',up)
     

Lower Limit: 5898584.319836798
Upper Limit: 10776739.708734632


##### Lower Limit (Batas Bawah)
Nilai ini (5898584.319836798) menunjukkan batas bawah dari rentang perkiraan interval keyakinan. Dengan tingkat kepercayaan 95%, kita memiliki keyakinan bahwa nilai sebenarnya dari hasil perkalian 'harga' dan 'terjual' kemungkinan besar berada di atas nilai ini, yang artinya nilai pendapatan terbawah dari pendapatan kotor yang mungkin didapat dari penjualan seblak di Tokopedia adalah sebesar 5.898.584

##### Upper Limit (Batas Atas)
Nilai ini (10776739.708734632) menunjukkan batas atas dari rentang perkiraan interval  keyakinan. Dengan tingkat kepercayaan 95%, kita memiliki keyakinan bahwa nilai sebenarnya dari hasil perkalian 'harga' dan 'terjual' kemungkinan besar berada di bawah nilai ini, yang artinya nilai pendapatan terbesar dari pendapatan kotor yang mungkin didapat dari penjualan seblak di Tokopedia adalah sebesar 10.776.739 


3. Analysis perbandingan harga seblak dari Jabodetabek dan diluar Jabodetabek.

Pada Analysis ini digunakan uji T untuk mendapatkan perbandingan harga produk seblak pada Jabodetabek dan diluar jabodetabek yang merupakan variabel independent dan bersifat numerik. Dalam kasus ini dua kelompok yang tidak terkait satu sama lain, yaitu harga barang di Jabodetabek dan di luar Jabodetabek. Terdapat pula varian yang homogen dalam kasus ini yaitu harga produk seblak di Jabodetabek dan diluar Jabodetabek dengan asumsi varians dalam dua kelompok ini (harga) relatif sama oleh karena itu dipilih lah uji dengan menggunkakan uji T 

In [15]:
analysis_data_seblak['area'] = np.where(analysis_data_seblak['lokasi'].str.contains('Jakarta|Tangerang|Depok|Bogor|Bekasi', case=False), 'JABODETABEK', 'NOT JABODETABEK')
jabodetabek = analysis_data_seblak[analysis_data_seblak['area'] == 'JABODETABEK']
non_jabodetabek = analysis_data_seblak[analysis_data_seblak['area'] != 'JABODETABEK']

In [17]:
# H0 = Tidak terdapat perbedaan harga barang yang signifikan antara Jabodetabek dan di luar Jabodetabek
# H1 = Perbedaan harga barang yang signifikan antara Jabodetabek dan di luar Jabodetabek

t_stat, p_val = stats.ttest_ind(jabodetabek['harga'],non_jabodetabek['harga'])
print('T-Statistic:',t_stat)
print('P-value:',p_val)

T-Statistic: -0.7282296623535762
P-value: 0.46696273308268865


T-Statistik (T-Statistic) 

Nilai ini (-0.7282296623535762) merupakan hasil  uji t independen.
 uji-t mengukur seberapa berbeda rata-rata dua kelompok  satu sama lain dalam satuan deviasi standar.
 Nilai negatif berarti rata-rata harga Jabodetabek cenderung lebih rendah dibandingkan rata-rata harga non-Jabodetabek, namun nilai tersebut belum cukup besar untuk dianggap signifikan.
 Nilai t yang mendekati nol menunjukkan bahwa perbedaan  kedua kelompok tidak signifikan..


 P-value (Nilai p): 
P-value (0.46696273308268865) adalah probabilitas mendapatkan hasil observasi atau yang lebih ekstrem jika hipotesis null benar. Dalam konteks ini, hipotesis null menyatakan bahwa tidak ada perbedaan signifikan antara rata-rata harga barang di Jabodetabek dan non-Jabodetabek. P-value yang lebih besar dari tingkat signifikansi yang umumnya digunakan (misalnya 0.05) menunjukkan bahwa tidak ada cukup bukti untuk menolak hipotesis null. Oleh karena itu, hasil tidak dianggap signifikan, dan kita tidak memiliki cukup bukti untuk menyatakan bahwa rata-rata harga barang di Jabodetabek berbeda secara signifikan dari non-Jabodetabek.

**Maka kesimpulannya H 0 gagal ditolak**

4. Menentukan preferensi Pembeli dari segi Harga.

Pada kasus ini saya ingin melihat apakah pembeli produk seblak di Tokopedia cenderung lebih menyukai produk dengan harga yang murah atau tidak, untuk menguji ini saya mencari korelasi antara harga dengan preferensi pembeli yang ditunjukkan dengan nilai rating yang ada pada Tokopedia. Saya akan menggunakan uji Spearman untuk mencari korelasi dari dua variabel ini karena saya mengasumsikan adanya hubungan monotone yang terjadi antara variabel rating dengan harga namun belum bisa dipastikan apakah hubungan monotone itu bersigat linier apa bukan dan secara asumsi yang ada akan berkorelasi secara negatif

In [23]:
#perhitungan spearman

corr_coef, p_value = stats.spearmanr(analysis_data_seblak['harga'], analysis_data_seblak['terjual']) 
print('Spearman: ', corr_coef)
print('P-value: ' , p_val)

Spearman:  -0.586269867612149
P-value:  0.46696273308268865


Nilai korelasi Spearman berkisar antara -1 dan 1 dan menunjukkan kekuatan dan arah hubungan antara dua variabel dalam skala ordinal atau nominal. Nilai yang lebih tinggi dari 1 menunjukkan hubungan yang kuat dan positif, nilai yang lebih rendah menunjukkan hubungan yang kuat dan negatif, dan nilai yang lebih rendah menunjukkan tidak adanya hubungan. Terlihat pada hasil adanya hubungan negatif yang moderat antara harga produk dan rating yang ditunjukkan oleh korelasi Spearman sebesar (-0.586), yang menunjukkan bahwa jumlah terjual produk cenderung meningkat seiring dengan menurunnya harganya. Namun, apabila dilihat dari nilai p -valuenya (0.4669), P-value menunjukkan angka yang lebih besar dari ambang standar p-value yang menunjukkan signifikansi (0.05) maka apabila dilihat dar p-value data tidak bisa dinyatakan bahwa harga berpengaruh terhadap rating secara signifikan. 

**Kesimpulan : walaupun dari nilai spearman, dapat dilihat bahwa adanya korelasi antara jumlah pembelian dari pembeli seblak dengan harga yang ditawarkan, namun nilai p-value menunjukkan tidak ada hubungan yang signifikan antara 2 variabel tersebut. Kesimpulan yang bisa diambil adalah bahwa HARGA BERKORELASI DENGAN RATING pada data penjualan seblak namun TIDAK CUKUP SIGNIFIKAN. Terdapat beberapa variabe atau faktor lain yang bisa mempengaruhi preferensi atau rating dari pembeli seblak di Tokopedia**

E. Conclusion

Dapat dikatakan bisnis dropshipper untuk berjualan seblak masih cukup menghasilkan apabila bisa memanfaatkan faktor luar secara maksimal untuk mendapatkan pembelian, dilihat dari range pendapatan yang dapat diambil yaitu dari range pendapatan minimum di angka  5.898.584 hingga maksimal di angka 10.776.739. Bukan angka yang mustahil untuk mendapatkan pendapatan tambahan sebesar 100.000 rupiah dari hasil penjualan seblak secara bersih, mengingat pereferensi pembeli untuk membeli produk seblak di Tokopedia masih cukup tinggi dengan jumlah pembelian rata rata  1126 pada produk. Pemilihan produsen atau supplier seblak di Tokopedia merupakan hal yang cukup krusial apabila ingin menjalankan bisnis dropshipper, karena banyak parameter yang perlu dipertimbangkan dalam memilihnya. Parameter itu termasuk harga produk yang sudah dibuktikan dari uji korelasi spearman berkorelasi dengan kemungkinan preferensi untuk membeli. Namun, selain harga banyak hal yang perlu dipikirkan strateginya guna mengoptimalkan kecenderungan customer untuk membeli. Pemilihan supplier dengan rating, lokasi, dan jumlah terjual yang baik bisa menjadi faktor lain yang mendukung preferensi pembelian customer. Mengingat bisnis dropshipper adalah bisnis yang dapat dilakukan tanpa modal, dan punya peluang yang cukup baik, maka saya merasa menjalankan bisnis dropshipper seblak adalah hal yang pantas untuk dilakukan guna menambah pendapatan tambahan.